## Details about educational institutions in San Fransisco

In the current project we will be using the data file hosted at [DataSF](https://datasf.org/). The dataset contains details about various educational instituions in San fransisco. For our current study we will be using only the columns institution and address.

In [6]:
import pandas as pd

edu_df = pd.read_csv('https://data.sfgov.org/api/views/6adp-a5yv/rows.csv?accessType=DOWNLOAD')
edu_df.head()

,the_geom,institutio,campus,address,webaddress,phone,st_area_sh,st_length_
0,MULTIPOLYGON (((-122.41422959861103 37.7811707...,University of California Hastings College of Law,<Null>,100 McAllister St,www.uchastings.edu,4155654600,13049.073407,492.593768
1,MULTIPOLYGON (((-122.41529196158669 37.7812009...,University of California Hastings College of Law,<Null>,198 McAllister St,www.uchastings.edu,4155654600,22642.056685,604.439178
2,MULTIPOLYGON (((-122.41701008790089 37.7813667...,University of California Hastings College of Law,<Null>,376 Larkin St,www.uchastings.edu,4155654600,25762.162205,650.066091
3,MULTIPOLYGON (((-122.41054924969892 37.8078130...,Academy of Art University,NaN,2300 Stockton St,NaN,0,37469.166147,821.907182
4,MULTIPOLYGON (((-122.41521002465295 37.8034172...,Academy of Art University,NaN,701 Chestnut St,NaN,0,10364.131930,432.953369


In [9]:
import numpy as np

edu_df.drop(['the_geom', 'campus', 'webaddress', 'phone', 'st_area_sh', 'st_length_'], axis=1, inplace=True)
edu_df.rename(columns = {"institutio": "Institution", "address":"Address",}, inplace=True)

edu_df['Latitude'] = np.nan
edu_df['Longitude'] = np.nan

edu_df.head()

,Institution,Address,Latitude,Longitude
0,University of California Hastings College of Law,100 McAllister St,NaN,NaN
1,University of California Hastings College of Law,198 McAllister St,NaN,NaN
2,University of California Hastings College of Law,376 Larkin St,NaN,NaN
3,Academy of Art University,2300 Stockton St,NaN,NaN
4,Academy of Art University,701 Chestnut St,NaN,NaN


## Getting coordinates of institution

We will be using the OpenCageGeocode API to get the accurate location of the institution. More detail about the aPI can be found at [OpenCageData.com](https://opencagedata.com/)

In [10]:
from opencage.geocoder import OpenCageGeocode

for index in edu_df.index:
    location =  geocoder.geocode(edu_df.at[index, 'Address'] + ', San Francisco, CA, United States of America')
    if len(location)>0:
        edu_df.at[index, 'Latitude'] = location[0]['geometry']['lat']
        edu_df.at[index, 'Longitude'] = location[0]['geometry']['lng']
    sleep(1)
    
print("Coordinates Obtained")

edu_df.head()

Coordinates Obtained


,Institution,Address,Latitude,Longitude
0,University of California Hastings College of Law,100 McAllister St,37.780989,-122.413846
1,University of California Hastings College of Law,198 McAllister St,37.781438,-122.415239
2,University of California Hastings College of Law,376 Larkin St,37.781173,-122.416986
3,Academy of Art University,2300 Stockton St,37.807098,-122.410420
4,Academy of Art University,701 Chestnut St,37.803313,-122.415063


## Getting details of popular locations near the educational restaurants

We will beusing the foursquare API to get the details of the popular locations located near the above educational institutions. To achieve this objective we will be using the **explore** endpoint

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        
        sleep(1)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['id'],
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue id',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
edu_venues = getNearbyVenues(names=edu_df['Address'],
                                   latitudes=edu_df['Latitude'],
                                   longitudes=edu_df['Longitude']
                                  )

The dataset contains details about the venue such as venue name, venue id, venue coordinates, venue category etc. Venue id wil be used to get further detail about the venue from *foursquare* 

In [11]:
edu_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue id,Venue,Venue Latitude,Venue Longitude,Venue Category
0,100 McAllister St,37.780989,-122.413846,4f5d49a5e4b0695cbbe24694,The Strand,37.779888,-122.413138,Theater
1,100 McAllister St,37.780989,-122.413846,46d36f99f964a520594a1fe3,Orpheum Theatre,37.779315,-122.414790,Theater
2,100 McAllister St,37.780989,-122.413846,4a73812ff964a520a0dc1fe3,Alonzo King LINES Dance Center,37.780116,-122.412187,Dance Studio
3,100 McAllister St,37.780989,-122.413846,59b759e1646e387b4245f12a,Villon,37.780865,-122.412471,French Restaurant
4,100 McAllister St,37.780989,-122.413846,55da2db5498eb79ab95580cb,George and Lennie,37.781701,-122.415213,Coffee Shop


## Removing duplicates from the edu_venue dataframe

In [15]:
print(edu_venues.shape)

print("\n After Removing duplicates \n")

edu_venues.drop_duplicates(['Venue id'], keep='first', inplace=True)

print(edu_venues.shape)

(3421, 8)

 After Removing duplicates 

(1876, 8)


## Getting restaurant details from popular locations list

In [16]:
res_data = edu_venues[edu_venues['Venue Category'].str.contains('restaurant', case=False)]
print(res_data.shape)

res_data.head()

(412, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue id,Venue,Venue Latitude,Venue Longitude,Venue Category
3,100 McAllister St,37.780989,-122.413846,59b759e1646e387b4245f12a,Villon,37.780865,-122.412471,French Restaurant
8,100 McAllister St,37.780989,-122.413846,54c42de4498ee52a43a5ee44,The Flying Falafel,37.781323,-122.411138,Mediterranean Restaurant
11,100 McAllister St,37.780989,-122.413846,49d66c4ef964a520c95c1fe3,Morty's Delicatessen,37.781710,-122.415243,American Restaurant
18,100 McAllister St,37.780989,-122.413846,547a3a09498edded45384099,Golden Era Vegan,37.781495,-122.416822,Vegetarian / Vegan Restaurant
20,100 McAllister St,37.780989,-122.413846,5633e4a5498e430fc92b4b53,Yemen Kitchen,37.783372,-122.412736,Middle Eastern Restaurant


## Getting details about each restaurant (Including popularity statistics and facility details)

We will be the using the Venue id and venue endpoint for getting further details about each restaurant. The venue endpoint is a premium call and the free developer verified account has a limit of 500 calls per day.`

In [17]:
def getRestaurantDetails(names, vid):
    
    restaurant_list=[]
    
    
    for name, res_id in zip(names, vid):
       
        sleep(1)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
            res_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        
        # make the GET request
        try:
            result = requests.get(url).json()['response']['venue']
        except Exception:
            print(url)
            traceback.print_exc()
            continue
        
        res_pricetier = None
        res_reservations = None
        res_outdoor = None
        res_wifi = None
        res_delivery = None
        res_music = None
        res_rroom = None
        res_proom = None
        res_wheelchair = None
        res_parking = None
        
        for group in result['attributes']['groups']:
            try:
                if group['type'].lower() == 'price':
                    res_pricetier = group['items'][0]['priceTier']
                    continue
            except:
                pass
            
            try:
                if group['type'].lower() == 'reservations':
                    res_reservations = group['items'][0]['displayValue']
                    continue
            except:
                pass
            try:
                if group['type'].lower() == 'outdoorseating':
                    res_outdoor = group['items'][0]['displayValue']
                    continue
            except:
                pass
            try:
                if group['type'].lower() == 'wifi':
                    res_wifi = group['items'][0]['displayValue']
                    continue
            except:
                pass
            try:
                if group['type'].lower() == 'diningoptions':
                    res_delivery = group['items'][0]['displayValue']
                    continue
            except:
                pass
            try:
                if group['type'].lower() == 'music':
                    res_music = group['items'][0]['displayValue']
                    continue
            except:
                pass
            try:
                if group['type'].lower() == 'restroom':
                    res_rroom = group['items'][0]['displayValue']
                    continue
            except:
                pass
            try:
                if group['type'].lower() == 'privateroom':
                    res_proom = group['items'][0]['displayValue']
                    continue
            except:
                pass
            try:
                if group['type'].lower() == 'wheelchairaccessible':
                    res_wheelchair = group['items'][0]['displayValue']
                    continue
            except:
                pass
            try:
                if group['type'].lower() == 'parking':
                    if 'no' in group['summary'].lower():
                        res_parking = 'Not Avaliable'
                    else:
                        res_parking = 'Available'
                        
                    continue
            except:
                pass

        # return only relevant information for each nearby venue
        try:
            restaurant_list.append([
                name,  
                result['name'],
                result['id'],  
                result['categories'][0]['name'],
                result['likes']['count'],
                result['photos']['count'],
                result['rating'],
                result['tips']['count'],
                res_pricetier,
                res_reservations,
                res_outdoor,
                res_wifi,
                res_delivery,
                res_music,
                res_rroom,
                res_proom,
                res_wheelchair,
                res_parking])
        except:
            print(url)
            traceback.print_exc()
            continue
            
    nearby_venues = pd.DataFrame([item for item in restaurant_list])

    nearby_venues.columns = ['Neighborhood', 
            'Venue',
            'Venue id',
            'Venue Category',
            'Venue Likes',
            'Venue photos count',
            'Venue rating', 
            'Venue Tips',
            'Price Tier',
            'Reservations',
            'Outdoor',
            'Wifi',
            'Delivery',
            'Music',
            'Rest room',
            'Private room',
            'Wheel chair',
            'Parking']
        
    return(nearby_venues)

In [19]:
res_venues = getRestaurantDetails(names=res_data['Neighborhood'],
                             vid=res_data['Venue id'])

res_venues.head()

(401, 21)


,Neighborhood,Venue,Venue id,Venue Category,Venue Likes,Venue photos count,Venue rating,Venue Tips,Price Tier,Reservations,...,Wifi,Delivery,Music,Rest room,Private room,Wheel chair,Parking,Institution,Latitude,Longitude
0,100 McAllister St,Villon,59b759e1646e387b4245f12a,French Restaurant,119,135,8.8,14,3.0,NaN,...,NaN,Delivery,NaN,NaN,NaN,NaN,Available,University of California Hastings College of Law,37.780989,-122.413846
1,100 McAllister St,The Flying Falafel,54c42de4498ee52a43a5ee44,Mediterranean Restaurant,271,132,8.7,58,2.0,No,...,NaN,Delivery,NaN,NaN,NaN,NaN,NaN,University of California Hastings College of Law,37.780989,-122.413846
2,100 McAllister St,Morty's Delicatessen,49d66c4ef964a520c95c1fe3,American Restaurant,36,39,8.1,31,1.0,No,...,NaN,Delivery,NaN,NaN,NaN,NaN,NaN,University of California Hastings College of Law,37.780989,-122.413846
3,100 McAllister St,Golden Era Vegan,547a3a09498edded45384099,Vegetarian / Vegan Restaurant,96,102,8.3,28,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,University of California Hastings College of Law,37.780989,-122.413846
4,100 McAllister St,Yemen Kitchen,5633e4a5498e430fc92b4b53,Middle Eastern Restaurant,28,39,8.3,9,2.0,No,...,NaN,Take-out,NaN,NaN,NaN,NaN,NaN,University of California Hastings College of Law,37.780989,-122.413846


The dataframe contains the detail about the venue statistics and venue facilities. The columns in the datset are:

In [21]:
res_venues.columns

Index([u'Neighborhood', u'Venue', u'Venue id', u'Venue Category',
       u'Venue Likes', u'Venue photos count', u'Venue rating', u'Venue Tips',
       u'Price Tier', u'Reservations', u'Outdoor', u'Wifi', u'Delivery',
       u'Music', u'Rest room', u'Private room', u'Wheel chair', u'Parking',
       u'Institution', u'Latitude', u'Longitude'],
      dtype='object')

The columns 'Venue Likes', 'Venue photos count', 'Venue rating' and 'Venue Tips' contains the total number of likes, photos, ratings and comments. 'Price Tier' categorizes the price of the menu into different categories. 'Reservations', 'Outdoor', 'Wifi', 'Delivery', 'Music', 'Rest room', 'Private room' and 'Wheel chair' columns contain information about the type of facilities at the restaurant.

## The data collected will be transformed and analyzed using cluster analysis to solve the business problem.